In [1]:
%%time 
import numpy as np
import pandas as pd
import os
import math
import time
import sys
import itertools

pd.options.display.max_rows = 8

#path = r'E:\YM\SMU_Term4\C7_MAS\'
#os.chdir(path.replace(os.sep, '/'))
print(os.getcwd())


C:\Users\U\Documents\C7_MAS\MAS_project\Boston_Hubway_Codes
Wall time: 2.54 s


In [2]:
H1_df = pd.read_csv('BSS Status-based_45_timestep1min_rep10-table.csv', skiprows=6)

In [3]:
H1_df

,[run number],sim_timestep_minutes,num_trucks,duration_to_pick,duration_to_drop,truck_capacity,truck_speed,wait_for_demo?,rebalancing_method,wait_time,pct_station_capacity_to_drop,pct_station_capacity_to_pick,truck_movement,shortage_threshold,days_to_simulate,[step],simulated_out_of_bikes_count,simulated_out_of_docks_count,sim_elapsed_hours
0,1,1,1,5,5,20,0.27,False,Status-based,0.05,30,30,Asynchronized,3,7,10078,313,44,168
1,2,1,2,5,5,20,0.27,False,Status-based,0.05,30,30,Asynchronized,3,7,10078,127,18,168
2,3,1,3,5,5,20,0.27,False,Status-based,0.05,30,30,Asynchronized,3,7,10078,114,10,168
3,4,1,4,5,5,20,0.27,False,Status-based,0.05,30,30,Asynchronized,3,7,10078,165,19,168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,447,1,2,5,5,20,0.27,False,Status-based,0.05,70,70,Asynchronized,3,7,10078,551,123,168
447,448,1,3,5,5,20,0.27,False,Status-based,0.05,70,70,Asynchronized,3,7,10078,475,143,168
448,449,1,4,5,5,20,0.27,False,Status-based,0.05,70,70,Asynchronized,3,7,10078,618,178,168
449,450,1,5,5,5,20,0.27,False,Status-based,0.05,70,70,Asynchronized,3,7,10078,627,220,168


In [4]:
H2_df = pd.read_csv('BSS Forecast-based_5_timestep1min_rep10-table.csv', skiprows=6)

In [5]:
H2_df

,[run number],sim_timestep_minutes,num_trucks,duration_to_drop,duration_to_pick,truck_capacity,truck_speed,wait_for_demo?,rebalancing_method,wait_time,pct_station_capacity_to_drop,pct_station_capacity_to_pick,truck_movement,shortage_threshold,days_to_simulate,[step],simulated_out_of_bikes_count,simulated_out_of_docks_count,sim_elapsed_hours
0,1,1,1,5,5,20,0.27,False,Forecast-based,0.05,50,50,Asynchronized,3,7,10078,1454,835,168
1,2,1,1,5,5,20,0.27,False,Forecast-based,0.05,50,50,Asynchronized,3,7,10078,1598,797,168
2,3,1,1,5,5,20,0.27,False,Forecast-based,0.05,50,50,Asynchronized,3,7,10078,1808,1111,168
3,4,1,1,5,5,20,0.27,False,Forecast-based,0.05,50,50,Asynchronized,3,7,10078,1803,874,168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,47,1,5,5,5,20,0.27,False,Forecast-based,0.05,50,50,Asynchronized,3,7,10078,1352,678,168
47,48,1,5,5,5,20,0.27,False,Forecast-based,0.05,50,50,Asynchronized,3,7,10078,1464,626,168
48,49,1,5,5,5,20,0.27,False,Forecast-based,0.05,50,50,Asynchronized,3,7,10078,1410,748,168
49,50,1,5,5,5,20,0.27,False,Forecast-based,0.05,50,50,Asynchronized,3,7,10078,1472,716,168


In [6]:
H2_df['pct_station_capacity_to_drop'] = np.NaN
H2_df['pct_station_capacity_to_pick'] = np.NaN

In [7]:
H3_df = pd.read_csv("BSS DisableRebalancing_5_timestep1min_rep10-table.csv", skiprows=6)

In [8]:
H3_df

,[run number],sim_timestep_minutes,num_trucks,duration_to_drop,duration_to_pick,truck_capacity,truck_speed,wait_for_demo?,rebalancing_method,wait_time,pct_station_capacity_to_drop,pct_station_capacity_to_pick,truck_movement,shortage_threshold,days_to_simulate,[step],simulated_out_of_bikes_count,simulated_out_of_docks_count,sim_elapsed_hours
0,1,1,1,5,5,20,0.27,False,Disable,0.05,50,50,Asynchronized,3,7,10078,1410,840,168
1,2,1,2,5,5,20,0.27,False,Disable,0.05,50,50,Asynchronized,3,7,10078,1725,807,168
2,3,1,3,5,5,20,0.27,False,Disable,0.05,50,50,Asynchronized,3,7,10078,1524,707,168
3,4,1,4,5,5,20,0.27,False,Disable,0.05,50,50,Asynchronized,3,7,10078,1544,848,168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,47,1,2,5,5,20,0.27,False,Disable,0.05,50,50,Asynchronized,3,7,10078,1682,881,168
47,48,1,3,5,5,20,0.27,False,Disable,0.05,50,50,Asynchronized,3,7,10078,1390,791,168
48,49,1,4,5,5,20,0.27,False,Disable,0.05,50,50,Asynchronized,3,7,10078,1437,731,168
49,50,1,5,5,5,20,0.27,False,Disable,0.05,50,50,Asynchronized,3,7,10078,1742,1029,168


In [9]:
H3_df['pct_station_capacity_to_drop'] = np.NaN
H3_df['pct_station_capacity_to_pick'] = np.NaN

In [10]:
H_df = H1_df.append(H2_df, sort=False)

In [11]:
H_df = H_df.append(H3_df, sort=False)

In [12]:
H_df

,[run number],sim_timestep_minutes,num_trucks,duration_to_pick,duration_to_drop,truck_capacity,truck_speed,wait_for_demo?,rebalancing_method,wait_time,pct_station_capacity_to_drop,pct_station_capacity_to_pick,truck_movement,shortage_threshold,days_to_simulate,[step],simulated_out_of_bikes_count,simulated_out_of_docks_count,sim_elapsed_hours
0,1,1,1,5,5,20,0.27,False,Status-based,0.05,30.0,30.0,Asynchronized,3,7,10078,313,44,168
1,2,1,2,5,5,20,0.27,False,Status-based,0.05,30.0,30.0,Asynchronized,3,7,10078,127,18,168
2,3,1,3,5,5,20,0.27,False,Status-based,0.05,30.0,30.0,Asynchronized,3,7,10078,114,10,168
3,4,1,4,5,5,20,0.27,False,Status-based,0.05,30.0,30.0,Asynchronized,3,7,10078,165,19,168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,47,1,2,5,5,20,0.27,False,Disable,0.05,NaN,NaN,Asynchronized,3,7,10078,1682,881,168
47,48,1,3,5,5,20,0.27,False,Disable,0.05,NaN,NaN,Asynchronized,3,7,10078,1390,791,168
48,49,1,4,5,5,20,0.27,False,Disable,0.05,NaN,NaN,Asynchronized,3,7,10078,1437,731,168
49,50,1,5,5,5,20,0.27,False,Disable,0.05,NaN,NaN,Asynchronized,3,7,10078,1742,1029,168


In [13]:
H_df.to_csv('BSS_Combined.csv', index=False)